<a href="https://colab.research.google.com/github/TAKE-JP-17/Pytorch/blob/main/World.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import ee
import geemap
import datetime
import calendar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
ee.Authenticate()
ee.Initialize(project='ee-f44081050')
map = geemap.Map()
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [3]:
#To draw User defined AOI
shp = map.user_roi
#shp = ee.FeatureCollection('projects/ee-p66121061/assets/drive-download-20240802T030740Z-001')

In [4]:
# Apply Cloud Masking & Scale factor

def maskL5(image):
  # Bit 0 - Fill
  # Bit 1 - Dilated Cloud
  # Bit 2 - Cirrus
  # Bit 3 - Cloud
  # Bit 4 - Cloud Shadow
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)

# Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B6').multiply(0.00341802).add(149.0)

# Replace the original bands with the scaled ones and apply the masks.
    return image.addBands(opticalBands, None, True) \
        .addBands(thermalBands, None, True) \
        .updateMask(qaMask) \
        .updateMask(saturationMask)

def maskL8(image):
  # Bit 0 - Fill
  # Bit 1 - Dilated Cloud
  # Bit 2 - Cirrus
  # Bit 3 - Cloud
  # Bit 4 - Cloud Shadow
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)

# Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)

# Replace the original bands with the scaled ones and apply the masks.
    return image.addBands(opticalBands, None, True) \
        .addBands(thermalBands, None, True) \
        .updateMask(qaMask) \
        .updateMask(saturationMask)

In [ ]:
# Create a list Landsat 5, 4 monthly
coll1 = []

# Initial date
Date1 = datetime.date(2000, 1, 1)
# Final date
Date = datetime.date(2022, 12, 31)

# Months interval (To compute n monthly composite image, set K = n-1)
k = 3
while Date1 <= Date:
    outyr = Date1.year + (Date1.month - 1 + k) // 12
    outmonth = (Date1.month - 1 + k) % 12 + 1

    # Adjust the month and year
    if Date1.month + k > 12:
        outyr = Date1.year + 1
        outmonth = (Date1.month + k) % 12

    # Adjust the day to the last day of the month
    outday = calendar.monthrange(outyr, outmonth)[1]

    Date2 = datetime.date(outyr, outmonth, outday)

    iniDate = str(Date1)
    endDate = str(Date2)

    dataset = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
        .filterBounds(shp) \
        .filterDate(iniDate, endDate) \
        .map(maskL5)

    # select one image from collection to copy its properties to the composite image
    img1 = dataset.first()

    # Get Composite image date & season
    x = str(Date1.year)
    y = int(Date1.month)

    # Change according to months used to generate composite image
    Jan_apr = [1, 2, 3, 4]
    May_aug = [5, 6, 7, 8]
    sep_dec = [9, 10, 11, 12]

    if y in Jan_apr:
        s = str("Jan_apr")
    elif y in May_aug:
        s = str("May_aug")
    elif y in sep_dec:
        s = str("sep_dec")

    season = s + x

    size = dataset.size().getInfo()
    if size >= 1:
        composite = dataset.median().copyProperties(img1, ['system:time_start'])
        coll1.append(composite)

        print(season + "-" + str(size) + "__" + iniDate + "_" + endDate)
    else:
        print("No image in " + season)

    # Update Date1 for the next iteration
    Date1 = Date2 + datetime.timedelta(days=1)

collection_L5 = ee.ImageCollection.fromImages(coll1)